In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import models, layers
from keras.utils import np_utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
sample_submission.head()

In [ ]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_df.head()

In [ ]:
missing_value_counts = train_df.isnull().sum()
total_missing_values = missing_value_counts.sum()
total_missing_values

Label - которые на цифры разбивают выделяем как отдельный класс

In [ ]:
print(train_df.info())

In [ ]:
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test_df.head()

In [ ]:
print(test_df.info())
X = train_df.drop("label", axis=1)
y = train_df.loc[:, "label"]

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(X, y, test_size=0.2)

In [ ]:
train_images = np.array(train_x)
train_images = train_images.reshape(len(train_images), 28,28,1)
train_images = train_images.astype("float32")
train_images = train_images / 255
train_labels = np.array(train_y)



In [ ]:
val_images = np.array(val_x)
val_images = val_images.reshape(len(val_images), 28,28,1)
val_images = val_images.astype("float32")
val_images = val_images / 255
val_labels = np.array(val_y)

In [ ]:
test_images = np.array(test_df)
test_images = test_images.reshape(len(test_images), 28,28,1)
test_images = test_images.astype("float32")
test_images = test_images / 255

In [ ]:
fig, axes = plt.subplots(5,5, figsize=(15, 15))
axes = axes.ravel()

for i in np.arange(0, 25):
    axes[i].imshow(train_images[i], cmap="Greys")
    axes[i].set_title("Digit = %s" % train_labels[i])
    axes[i].axis("off")
    plt.subplots_adjust(wspace=1)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
input_shape=(28,28,1)

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), padding="same", activation="relu", input_shape=(28,28,1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(rate=0.3))
model.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(rate=0.3))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(10, activation="softmax"))
model.summary()



In [ ]:


model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [ ]:
batch_size = 264


early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(train_images, train_labels, epochs=15, batch_size=batch_size,
                    verbose=1,validation_data=(val_images, val_labels),callbacks=[early_stopping])

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
#history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

In [ ]:
predictions = model.predict(test_images)
prediction_classes = np.argmax(predictions, axis=1)

In [ ]:
output = pd.DataFrame({"ImageId": sample_submission.ImageId, "label": prediction_classes})
output.to_csv("My_submission_end_next day.csv", index=False)
output.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split


log_clf = LogisticRegression ( ) 
rnd_clf = RandomForestClassifier ( )
svm_clf = SVC()


#from sklearn.datasets import make_moons
#X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X_train,y_train,x_test,y_test
voting_clf = VotingClassifier (
estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting= 'hard' )
voting_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
index = sample_submission.ImageId
data = {'ImageId' : index,'Label': test_pred}
df = pd.DataFrame(data)
df.head
df.to_csv('submission2.csv', index=False)